In [1]:
%load_ext autoreload
%autoreload 2

#  Sliceguard – Find critical data segments in your data (fast)

Sliceguard is a python library for quickly finding **critical data slices** like outliers, errors, or biases. It works on **structured** and **unstructured** data.

This notebook showcases especially the unstructured data case, so if you have tabular data instead have a look at [this notebook](examples/quickstart_structured_data.ipynb) instead

It is interesting for you if you want to do the following:
1. Find **performance issues** of your machine learning model.
2. Find **anomalies and inconsistencies** in your data.
3. Quickly **explore** your data using an interactive report to generate **insights**.

To run this notebook install and import sliceguard:

In [ ]:
!pip install sliceguard

In [2]:
from sliceguard import SliceGuard
from sliceguard.data import from_huggingface

Now download the demo dataset with our utility function:

In [26]:
df = from_huggingface("Matthijs/snacks")

You now have the following dataframe containing an image column with a path to the raw image on the harddrive, a label and a split marker.

In [27]:
df

,image,label,split
0,/home/daniel/.cache/huggingface/datasets/downl...,apple,train
1,/home/daniel/.cache/huggingface/datasets/downl...,apple,train
2,/home/daniel/.cache/huggingface/datasets/downl...,apple,train
3,/home/daniel/.cache/huggingface/datasets/downl...,apple,train
4,/home/daniel/.cache/huggingface/datasets/downl...,apple,train
...,...,...,...
950,/home/daniel/.cache/huggingface/datasets/downl...,watermelon,validation
951,/home/daniel/.cache/huggingface/datasets/downl...,watermelon,validation
952,/home/daniel/.cache/huggingface/datasets/downl...,watermelon,validation
953,/home/daniel/.cache/huggingface/datasets/downl...,watermelon,validation


## Check for larger error groups and outliers

In [57]:
sg = SliceGuard()
issues = sg.find_issues(df, ["image"], drop_reference="overall") # also try out drop_reference="parent" for more class-specific results

Feature image was inferred as referring to raw data. If this is not the case, please specify in feature_types!
Using default model for computing embeddings for feature image.


/home/daniel/code/sliceguard/.venv/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


Embedding computation on cuda with batch size 1 and multiprocessing None.


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/6745 [00:00<?, ? examples/s]

In [56]:
report_df = sg.report()

(                                                 image       label  \
 0    /home/daniel/.cache/huggingface/datasets/downl...       apple   
 1    /home/daniel/.cache/huggingface/datasets/downl...       apple   
 2    /home/daniel/.cache/huggingface/datasets/downl...       apple   
 3    /home/daniel/.cache/huggingface/datasets/downl...       apple   
 4    /home/daniel/.cache/huggingface/datasets/downl...       apple   
 ..                                                 ...         ...   
 950  /home/daniel/.cache/huggingface/datasets/downl...  watermelon   
 951  /home/daniel/.cache/huggingface/datasets/downl...  watermelon   
 952  /home/daniel/.cache/huggingface/datasets/downl...  watermelon   
 953  /home/daniel/.cache/huggingface/datasets/downl...  watermelon   
 954  /home/daniel/.cache/huggingface/datasets/downl...  watermelon   
 
           split                                       sg_emb_image  sg_y_pred  
 0         train  [0.12190916389226913, 0.2230367809534073, -0.1.

## Let sliceguard train a model to pinpoint problems even better

In [ ]:
sg = SliceGuard()
issues = sg.find_issues(df.sample(1000), ["image"], y="label", drop_reference="overall") # also try out drop_reference="parent" for more class-specific results

## Train an own advanced model and find its weaknesses